In [1]:
# needed library

import sys, os
import numpy as np
from collections import OrderedDict

import import_ipynb
from layers_collection import *

class neural_network:
    
    def __init__(self, input_size, hidden_size, output_size, 
                 activation='relu', weight_init_std = 'relu', weight_decay_lambda=0,
                 dropout = False, dropout_p = 0.3, batchnorm = False):
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.hidden_layer = len(hidden_size)
        self.output_size = output_size
        
        self.dropout = dropout
        self.batchnorm = batchnorm
        
        self.weight_decay_lambda = weight_decay_lambda
        self.params = {}
        
        self.weight_init(weight_init_std)
        
        activation_layer = {
            'relu' : Relu,
            'sigmoid' : Sigmoid
        }
        
        self.layers = OrderedDict()
        
        for i in range(1, self.hidden_layer+1):
            self.layers['Affine'+str(i)] = Affine(self.params['W'+str(i)], 
                                                  self.params['b'+str(i)])
            
            if self.batchnorm:
                self.params['gamma'+str(i)] = np.ones(hidden_size[i-1])
                self.params['beta'+str(i)] = np.zeros(hidden_size[i-1])
                self.layers['BN'+str(i)] = BatchNormalization(self.params['gamma'+str(i)],
                                                              self.params['beta'+str(i)])
            
            self.layers['activation function'+str(i)] = activation_layer[activation]()
            
            if self.dropout:
                self.layers['Dropout'+str(i)] = Dropout(dropout_p)
        
        i = self.hidden_layer + 1
        self.layers['Affine'+str(i)] = Affine(self.params['W'+str(i)], self.params['b'+str(i)])
        self.last_layer = SoftmaxWithLoss()
    
    def weight_init(self, weight_init_std):
        total_size = [self.input_size] + self.hidden_size + [self.output_size]
        for i in range(1, len(total_size)):
            
            scale = weight_init_std
            
            if weight_init_std == 'relu':
                scale = np.sqrt(2.0/total_size[i-1])
            elif weight_init_std == 'sigmoid':
                scale = np.sqrt(1.0 / total_size[i - 1]) 
            
            self.params['W'+str(i)] = scale * np.random.randn(total_size[i-1], total_size[i])
            self.params['b'+str(i)] = np.zeros(total_size[i])
            
    def predict(self, x, flg = False):
        for k, layer in self.layers.items():
            if 'Dropout' in k or 'BN' in k:
                x = layer.forward(x, flg)
            else:
                x = layer.forward(x)
        
        return x

    def loss(self, x, t, flg=False):
        y = self.predict(x, flg)
        
        weight_decay = 0
        for i in range(1,self.hidden_layer+2):
            W = self.params['W'+str(i)]
            weight_decay += 0.5 * self.weight_decay_lambda * np.sum(W**2)
        
        return self.last_layer.forward(y,t) + weight_decay
        

    def accuracy(self, x, t):
        y = self.predict(x, flg=False)
        y = np.argmax(y, axis=1)
        if t.ndim != 1:
            t = np.argmax(t,axis=1)
        accuracy = np.sum(y == t)/ float(x.shape[0])
        return accuracy
    
    def gradient(self, x, t):

        self.loss(x, t, flg=True)

        dout = 1
        dout = self.last_layer.backward(dout)

        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        grads = {}
        for i in range(1, self.hidden_layer+2):
            grads['W' + str(i)] = self.layers['Affine' + str(i)].dW + self.weight_decay_lambda * self.params['W' + str(i)]
            grads['b' + str(i)] = self.layers['Affine' + str(i)].db
            
            if self.batchnorm and i != self.hidden_layer+1:
                grads['gamma' + str(i)] = self.layers['BN' + str(i)].dgamma
                grads['beta' + str(i)] = self.layers['BN' + str(i)].dbeta

        return grads

C:\ana3\envs\t_f2.2\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\ana3\envs\t_f2.2\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\ana3\envs\t_f2.2\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


importing Jupyter notebook from layers_collection.ipynb
